##  Crash Severity Classification

### General Parameters

In [ ]:
# List of years used to train, test, and validate the model:
year_filter = [2011]

# Set to True to save model after each epoch:
save_model = False

# Set to True to save plots for each epoch:
save_plots = False

# Set to True to stop straining process after overfitting:
Early_stopping = False

# Set to True to set label_index automatically:
automatic_labelling = False

# Name of the model for saving and logging purposes:
model_name = "2011Narr"

# Save location:
address = "/content/gdrive/My Drive/Modelling Results"

# The dataset csv file address:
dataset_address = 'Crash Severity Data.csv'

### Import dataset from Google Drive

In [ ]:
# If the dataset is being loaded from the google drive (Uncomment and replace the file id):
# !gdown --id <Google drive file id of the dataset file>

### Install libraries

In [ ]:
# Comment out if you have the libraries installed on your device:
%pip install pytorch_transformers
%pip install -U -q PyDrive
%pip install seaborn
%pip install logging
%pip install numpy
%pip install pandas
%pip install tqdm
%pip install scikit-learn
%pip install torch
%pip install pytorch-transformers
%pip install matplotlib

### Set things up

In [ ]:
# Basic Utilities:
import os
import re
import json
import logging
from itertools import cycle

# Data Utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# Report Utilities:
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

# Neural Network Related:
import torch
import torch.nn.functional as nnf
from torch.utils.data import TensorDataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule

# Mounting google drive (to be able to save logging info in Google Drive):
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


### BERT Input class

In [ ]:
class BertInputItem(object):

    def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
        
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


#### Input Conversion Function

In [ ]:
def convert_samples_to_inputs(sample_texts, sample_labels, label2idx, max_seq_length, tokenizer):
    """converts data to BERT compatible input

    Args:
        sample_texts (Pandas Dataframe): Crash text narrative data (X)
        sample_labels (Pandas Dataframe): Crash type label data (y)
        label2idx (dict): label index pairs
        max_seq_length (int): Number of tokens used to perform prediction (3 to 512)
        tokenizer (function): Pretrained BERT tokenizer

    Returns:
        List: list of BERT input items
    """
    
    input_items = []
    examples = zip(sample_texts, sample_labels)
    for (ex_index, (text, label)) in enumerate(examples):

        # Create a list of token ids
        input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
        if len(input_ids) > max_seq_length:
            input_ids = input_ids[:max_seq_length]
            input_ids[max_seq_length-1] = tokenizer.encode(f"[SEP]")[0]

        # All our tokens are in the first input segment (id 0).
        segment_ids = [0] * len(input_ids)

        # The mask has 1 for real tokens and 0 for padding tokens.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label2idx[label]

        input_items.append(
            BertInputItem(text=text,
                          input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))
    
    return input_items


#### BERT input data loader

In [ ]:
def get_data_loader(features, batch_size, shuffle=True): 

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader

### Evaluation function

In [ ]:
def evaluate(model, dataloader):
    """Performs evaluation on the model

    Args:
        model: Trained model
        dataloader: Dataloader

    Returns:
        List: [evaluation loss, correct labels, predicted labels, probabilities]
    """

    model.eval()
    
    eval_loss = 0
    nb_eval_steps = 0
    predicted_labels, correct_labels, all_probs = [], [], []
    logits = None

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        with torch.no_grad():
            tmp_eval_loss, logits = model(input_ids, attention_mask=input_mask,
                                          token_type_ids=segment_ids, labels=label_ids)

        probs = nnf.softmax(logits.to('cpu'), dim=1)
        outputs = np.argmax(logits.to('cpu'), axis=1)
        label_ids = label_ids.to('cpu').numpy()
        
        predicted_labels += list(outputs)
        correct_labels += list(label_ids)
        all_probs += list(probs)
        
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    
    correct_labels = np.array(correct_labels)
    predicted_labels = np.array(predicted_labels)

    probs = nnf.softmax(logits, dim=1) # assuming logits has the shape [batch_size, nb_classes]
        
    return eval_loss, correct_labels, predicted_labels, all_probs

### Report Functions

In [ ]:
def save_report_json(actual_labels,
                     predicted_labels,
                     iter_number=None,
                     dataset="train",
                     model_name="temporary tests",
                     address="Amir/Crash_Narr/GEN 1"):
    
    report = classification_report(actual_labels, predicted_labels, output_dict=True)
    
    temp = dict()
    if not os.path.exists(f'{address}/Model Reports/{model_name}.json'):
        with open(f'{address}/Model Reports/{model_name}.json', 'w') as f:
            json.dump(temp,f)
    else:
        with open(f'{address}/Model Reports/{model_name}.json', 'r') as f:
            temp = json.loads(f.read())
            
    if str(iter_number) not in temp: temp[str(iter_number)] = dict()
    temp[str(iter_number)][dataset]= report
    
    with open(f'{address}/Model Reports/{model_name}.json', "w") as f:
        json.dump(temp,f)
    
    return report



#### Confusion Matrix Plot Function

In [ ]:
def plot_confusion(correct_data, predicted_data, labels, data_type = "Test", figure_name="epoch"):
    cf_matrix = confusion_matrix(correct_data, predicted_data)
    
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 28})
    ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
    
    ax.set_title(f'{data_type} Confusion Matrix\n\n');
    ax.set_xlabel('Predicted Crash Type\n', labelpad=30)
    ax.set_ylabel('\nActual Crash Type', labelpad=40);
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)
    if save_plots:
        plt.savefig(f"{address}/Plots/{model_name}-confusion-{data_type}-{figure_name}.png", dpi=150)
    plt.show()
    print()
    
    return cf_matrix

#### Accuracy/Loss Plot and Save Function

In [ ]:
def plot_epochs(dev_metric_data, train_metric_data, test_metric_data, metric_type="Loss", figure_name="epoch"):
    
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 28})
    epochs = range(NUM_TRAIN_EPOCHS)
    
    plt.plot(epochs, train_metric_data, 'g', label=f'Train {metric_type}', linewidth=3.0)
    plt.plot(epochs, dev_metric_data, 'b', label=f'Dev {metric_type}', linewidth=3.0)
    plt.plot(epochs, test_metric_data, 'r', label=f'Test {metric_type}', linewidth=3.0)
    plt.title(f'Training, Development and Test {metric_type}\n\n')
    plt.ylabel('\nLoss',labelpad=30)
    plt.xlabel('Epochs\n',labelpad=40)
    
    plt.xticks(list(epochs))
    if metric_type=="Loss":
        axes = plt.axes()
        axes.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        axes.set_ylim([0, 0.8])
    else:
        plt.yticks(list(range(80,101,4)))
        axes = plt.axes()
        axes.set_ylim([80, 100])
    
    plt.xticks(list(range(0,NUM_TRAIN_EPOCHS)))
    axes.set_xlim([0, NUM_TRAIN_EPOCHS-1])
    
    plt.legend()
    if save_plots:
        plt.savefig(f"{address}/Plots/{model_name}-{metric_type}-{figure_name}.png", dpi=150)
    plt.show()
    print()

    temp = dict()
    if not os.path.exists(f'{address}/Model Epochs/{model_name}.json'):
        with open(f'{address}/Model Epochs/{model_name}.json', 'w') as f:
            json.dump(temp,f)
    else:
        with open(f'{address}/Model Epochs/{model_name}.json', 'r') as f:
            temp = json.loads(f.read())
            
    if metric_type not in temp: temp[metric_type] = dict()
    temp[metric_type]['dev'] = dev_metric_data
    temp[metric_type]['train'] = train_metric_data
    temp[metric_type]['test'] = test_metric_data
    
    with open(f'{address}/Model Epochs/{model_name}.json', "w") as f:
        json.dump(temp,f)

    return temp


#### ROC Curves Plot function

In [ ]:
def plot_roc(y_correct, y_prob, data_type="Validation", figure_name="epoch"):
    
    inv_label2idx = {v: k for k, v in label2idx.items()}
    y_correct = y_correct.reshape(-1, 1)
    y_prob = np.array([j.numpy() for j in y_prob])
    
    onehotencoder = OneHotEncoder()
    y_correct= onehotencoder.fit_transform(y_correct).toarray()
    n_classes = y_prob.shape[1]
    
    # Plotting and estimation of FPR, TPR
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    plt.figure(figsize=(10,10))
    plt.title(f'{data_type} ROC Curves\n', fontsize=22)
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_correct[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        colors = cycle(['blue', 'green', 'red','darkorange','olive','purple','navy'])
    
    plt.rcParams.update({'font.size': 18})
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=1.5, label='ROC curve of class {0} (AUC = {1:0.2f})'.format(inv_label2idx[i], roc_auc[i]))
        plt.plot([0, 1], [0, 1], '--', lw=1, dashes=(5, 10))
        plt.xlim([-0.05, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', labelpad=30, fontsize = 18)
        plt.ylabel('True Positive Rate', labelpad=40, fontsize = 18)
        plt.legend(loc="lower right", prop={'size': 18})
        plt.tick_params(labelsize=18)
    
    ax = plt.axes()
    ax.set_ylim([0, 1])
    ax.set_xlim([0, 1])
    
    if save_plots:
        plt.savefig(f"{address}/Plots/{model_name}-ROC-{data_type}-{figure_name}.png", dpi=150)
    plt.show()

### Load Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

# Loading the dataset:
BERT_MODEL = "bert-base-uncased"
temp = pd.read_csv(dataset_address, encoding= 'unicode_escape')
data = temp.loc[temp["Year"].isin(year_filter)][:]
y_frame = data ["SEVERITY_CD"]
x_frame = data ["narr"]
x_frame = x_frame.apply(str)

# removing the extra spaces:
for i in range(len(x_frame)):
    x_frame.iat[i] = re.sub(' +', ' ', x_frame.iat[i])

# Specifying Label:Index pairs:
label2idx = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

if automatic_labelling:
    target_names = list(set(y_frame))
    label2idx = {label: idx for idx, label in enumerate(target_names)}

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = len(label2idx))
model.to(device)

# Train, Test, Validation split(Train size = 70%, Test size = 20%, Val size = 10%)
train_texts, rest_texts, train_labels, rest_labels = train_test_split(x_frame, y_frame, test_size=0.3, random_state=1)
test_texts, val_texts, test_labels, val_labels = train_test_split(rest_texts, rest_labels, test_size=0.33333, random_state=1)

print("Train size:", len(train_labels))
print("Test size:", len(test_labels))
print("Validation size:", len(val_labels))


MAX_SEQ_LENGTH = 512
BATCH_SIZE = 16

train_features = convert_samples_to_inputs(train_texts, train_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_samples_to_inputs(test_texts, test_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
val_features = convert_samples_to_inputs(val_texts, val_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)

train_dataloader = get_data_loader(train_features, BATCH_SIZE, shuffle=True)
test_dataloader = get_data_loader(test_features, BATCH_SIZE, shuffle=False)
val_dataloader = get_data_loader(val_features, BATCH_SIZE, shuffle=False)

### Hyper-parameters , optimizer & schedualer

In [ ]:
GRADIENT_ACCUMULATION_STEPS = 1
NUM_TRAIN_EPOCHS = 6
LEARNING_RATE = 5e-5
WARMUP_PROPORTION = 0.1
MAX_GRAD_NORM = 5

num_train_steps = int(len(train_dataloader.dataset) / BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(WARMUP_PROPORTION * num_train_steps)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, correct_bias=False)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_train_steps)

### Train model

In [14]:
# Early stopping patience:
PATIENCE = 2

OUTPUT_DIR = f'{address}/Model Saved Sates/'

if save_model:
    os.makedirs(OUTPUT_DIR, exist_ok = True)

train_history = []
loss_history = []
val_history = []

te_correct_history = []
te_predicted_history = []
tr_correct_history = []
tr_predicted_history = []
val_correct_history = []
val_predicted_history = []

epoch_number = 0
no_improvement = 0
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):

    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        outputs = model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids, labels=label_ids)
        loss = outputs[0]

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        tr_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)  
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
    
    model.eval()
    
    test_loss, te_correct, te_predicted, te_probabilities = evaluate(model, test_dataloader)
    save_report_json(te_correct, te_predicted, epoch_number, "test",
                     model_name=model_name, address=address)
    plot_confusion(te_correct, te_predicted, label2idx.keys(), "Test", figure_name=str(epoch_number))
    plot_roc(te_correct, te_probabilities, "Test", figure_name=str(epoch_number))

    # save model:
    if save_model:
        MODEL_FILE_NAME = f"{model_name}-{str(epoch_number)}.bin"
        model_to_save = model.module if hasattr(model, 'module') else model
        output_model_file = os.path.join(OUTPUT_DIR, MODEL_FILE_NAME)
        torch.save(model_to_save.state_dict(), output_model_file)

    # evaluate train and test loss:
    train_loss, tr_correct, tr_predicted, tr_probabilities = evaluate(model, train_dataloader)
    save_report_json(tr_correct, tr_predicted, epoch_number, "train",
                     model_name=model_name, address=address)
    plot_confusion(tr_correct, tr_predicted, label2idx.keys(), "Train", figure_name=str(epoch_number))
    plot_roc(tr_correct, tr_probabilities, "Train", figure_name=str(epoch_number))

    validation_loss, val_correct, val_predicted, val_probabilities = evaluate(model, val_dataloader)
    save_report_json(val_correct, val_predicted, epoch_number, "validation",
                     model_name=model_name, address=address)
    plot_confusion(val_correct, val_predicted, label2idx.keys(), "Validation", figure_name=str(epoch_number))
    plot_roc(val_correct, val_probabilities, "Validation", figure_name=str(epoch_number))
    
    print("Loss history:", loss_history)
    print("test loss:", test_loss)

    epoch_number += 1
    
    loss_history.append(test_loss)
    val_history.append(validation_loss)
    train_history.append(train_loss)

    te_correct_history.append(te_correct)
    te_predicted_history.append(te_predicted)
    tr_correct_history.append(tr_correct)
    tr_predicted_history.append(tr_predicted)
    val_correct_history.append(val_correct)
    val_predicted_history.append(val_predicted)
    
    if Early_stopping:
        if len(loss_history) == 0 or test_loss < min(loss_history):
            no_improvement = 0
        else:
            no_improvement += 1
        if no_improvement >= PATIENCE: 
            print("No improvement on development set. Finish training.")
            break


### Plot Accuracy & Loss

In [ ]:
te_accuracy = []
val_accuracy = []
tr_accuracy = []

for i,j in zip(te_correct_history, te_predicted_history):
    te_accuracy.append(np.mean(i == j)*100)
for i,j in zip(val_correct_history, val_predicted_history):
    val_accuracy.append(np.mean(i == j)*100)
for i,j in zip(tr_correct_history, tr_predicted_history):
    tr_accuracy.append(np.mean(i == j)*100)

print(plot_epochs(te_accuracy, tr_accuracy, val_accuracy, metric_type="Accuracy"))

print(plot_epochs(loss_history, train_history, val_history, metric_type="Loss"))
